In [208]:
from yfinance import download
from numpy import where, nan
from pandas import isna

# PREDIÇÃO DE DADOS BINÁRIOS

Predição da série dos retornos de ativos financeiros

In [144]:
ticker = 'GOAU4.SA'
start = '2012-05-12'
end = '2022-05-12'
column_variacao_percentual = 'Adj Close'
periodos_alvo = 1

In [ ]:
df_price = download(ticker, start, end, progress=False)
df_price.tail(2)

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA
Date,,,,,,
2022-05-10,8.179797,10.78,11.30,10.60,11.3,11194400
2022-05-11,8.498492,11.20,11.34,10.83,10.9,9818700


A ``variação percentual`` será utilizada para derivar variáveis de entrada que alimentam o modelo de aprendizado de máquina, auxiliando na identificação de padrões nos dados.

![alt text](Screenshot_1.png)

In [146]:
df_price['variacao_percentual'] = df_price[column_variacao_percentual].pct_change(periodos_alvo)
df_price.tail(2)

Price,Adj Close,Close,High,Low,Open,Volume,variacao_percentual
Ticker,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,
Date,,,,,,,
2022-05-10,8.179797,10.78,11.30,10.60,11.3,11194400,-0.054386
2022-05-11,8.498492,11.20,11.34,10.83,10.9,9818700,0.038961


Estou criando uma nova variável, `'alvo_numerico'`, que representa a diferença entre o preço de fechamento e o de abertura. Em seguida, desloca essa coluna para cima, preparando os dados para um modelo que utilizará a variação futura como "alvo" (o que o modelo tentará prever).

O último dado da coluna `'alvo_numerico'` é `NaN`, pois não existe um valor referente ao próximo dia para realizar o deslocamento.

In [147]:
df_price['alvo_numerico'] = df_price['Close'] - df_price['Open']
df_price['alvo_numerico'] = df_price['alvo_numerico'].shift(-periodos_alvo)
df_price.tail(3)

Price,Adj Close,Close,High,Low,Open,Volume,variacao_percentual,alvo_numerico
Ticker,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,,
Date,,,,,,,,
2022-05-09,8.650251,11.40,11.57,11.28,11.41,8711800,-0.020618,-0.52
2022-05-10,8.179797,10.78,11.30,10.60,11.30,11194400,-0.054386,0.30
2022-05-11,8.498492,11.20,11.34,10.83,10.90,9818700,0.038961,NaN


Ou seja, a previsão é realizada com base nos dados do dia anterior, e o alvo será o preço de fechamento do dia seguinte.



In [157]:
df_alvo_explicativo = df_price.copy()
df_alvo_explicativo = df_alvo_explicativo.reset_index()
df_alvo_explicativo['Date'] = df_alvo_explicativo['Date'].astype(str)
df_alvo_explicativo['variacao_numerica'] = df_alvo_explicativo['Close'] - df_alvo_explicativo['Open']
df_alvo_explicativo['alvo_numerico_e_data'] = (df_alvo_explicativo['variacao_numerica'].astype(str) + ' | ' + df_alvo_explicativo['Date'])
df_alvo_explicativo['alvo_numerico_e_data'] = df_alvo_explicativo['alvo_numerico_e_data'].shift(-periodos_alvo)
df_alvo_explicativo[['Date', 'Open', 'Close', 'variacao_percentual', 'alvo_numerico', 'alvo_numerico_e_data']].tail(3)


Price,Date,Open,Close,variacao_percentual,alvo_numerico,alvo_numerico_e_data
Ticker,,GOAU4.SA,GOAU4.SA,,,
2476,2022-05-09,11.41,11.40,-0.020618,-0.52,-0.5200004577636719 | 2022-05-10
2477,2022-05-10,11.30,10.78,-0.054386,0.30,0.3000001907348633 | 2022-05-11
2478,2022-05-11,10.90,11.20,0.038961,NaN,None


O cálculo do retorno real será feito com base na diferença entre o preço de abertura (Open) e o preço de fechamento (Close), em vez de utilizar a variação percentual.

Quando o valor de `alvo_numerico` para o próximo dia é `NaN` (por exemplo, quando não há dados disponíveis para o dia seguinte), eu considero que a previsão não é válida. Nesse caso, eu também defino o valor de `alvo_binario` como `NaN`, pois a previsão não pode ser feita adequadamente.

Se eu não fizer essa alteração, ao calcular o retorno real, o valor `NaN` no `np.where()` será considerado como `0`, o que pode gerar uma previsão errada ou inválida.

In [217]:
df_price['alvo_binario'] = where(df_price['alvo_numerico'] > 0, 1, 0)

if isna(df_price.iloc[-1, df_price.columns.get_loc('alvo_numerico')]).item():
    df_price.iloc[-1, df_price.columns.get_loc('alvo_numerico')] = nan
    df_price.iloc[-1, df_price.columns.get_loc('alvo_binario')] = nan

df_price.tail(3)

Price,Adj Close,Close,High,Low,Open,Volume,variacao_percentual,alvo_numerico,alvo_binario
Ticker,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,GOAU4.SA,,,
Date,,,,,,,,,
2022-05-09,8.650251,11.40,11.57,11.28,11.41,8711800,-0.020618,-0.52,0.0
2022-05-10,8.179797,10.78,11.30,10.60,11.30,11194400,-0.054386,0.30,1.0
2022-05-11,8.498492,11.20,11.34,10.83,10.90,9818700,0.038961,NaN,NaN
